In [1]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
df=pd.read_csv("/content/BATASHOE.csv",parse_dates=['DATE'])
df.head()

#       DATE TRADING CODE    LTP*  ...     YCP  TRADE  VALUE (mn)  VOLUME
0  439 2018-05-08     BATASHOE  1123.0  ...  1109.6   40.0        0.58   525.0
1  438 2018-06-08     BATASHOE  1101.0  ...  1109.6   46.0        1.34  1210.0
2  437 2018-07-08     BATASHOE  1108.0  ...  1102.5   24.0        0.60   540.0
3  436 2018-08-08     BATASHOE  1108.0  ...  1107.2    8.0        0.44   396.0
4  435 2018-09-08     BATASHOE  1109.9  ...  1108.3   26.0        3.34  3009.0

[5 rows x 12 columns]

In [2]:
# Sort DataFrame by date
df = df.sort_values('DATE')

# Double check the result
df.head()

#       DATE TRADING CODE    LTP*  ...     YCP  TRADE  VALUE (mn)  VOLUME
36  403 2018-01-10     BATASHOE  1092.0  ...  1094.9   22.0        0.50   451.0
59  380 2018-01-11     BATASHOE  1086.6  ...  1090.9   45.0        0.65   600.0
37  402 2018-02-10     BATASHOE  1093.0  ...  1095.9   17.0        0.26   240.0
79  360 2018-02-12     BATASHOE  1135.6  ...  1131.4   82.0        3.76  3303.0
16  423 2018-03-09     BATASHOE  1110.1  ...  1111.4    1.0        0.00     1.0

[5 rows x 12 columns]

In [3]:
df = df.set_index('DATE')
df.head(3)

# TRADING CODE    LTP*    HIGH  ...     YCP  TRADE  VALUE (mn)  VOLUME
DATE                                          ...                                   
2018-01-10  403     BATASHOE  1092.0  1105.0  ...  1094.9   22.0        0.50   451.0
2018-01-11  380     BATASHOE  1086.6  1095.0  ...  1090.9   45.0        0.65   600.0
2018-02-10  402     BATASHOE  1093.0  1104.0  ...  1095.9   17.0        0.26   240.0

[3 rows x 11 columns]

In [4]:
df['daily_ret'] = df['CLOSEP*'].pct_change()
df['daily_ret'] = df['daily_ret'].fillna(0.0)

In [5]:
df['volatility']= df['daily_ret'].rolling(window=21).std()
df['volatility'] = df['volatility'].fillna(0.0)

In [6]:
df['volatility'] = df['volatility']*100


In [7]:
import matplotlib.pyplot as plt 

In [8]:
df=df.reset_index()['volatility']

In [9]:
df.shape

(439,)

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(np.array(df).reshape(-1,1))

In [12]:
##splitting dataset into train and test split where train=70 percent and test=30 percent
training_size=int(len(df)*0.70)
test_size=len(df)-training_size
train_data,test_data=df[0:training_size,:],df[training_size:len(df),:1]




In [13]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [],[]
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [14]:

# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [15]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
import xgboost as xgb
from sklearn.datasets import make_regression



In [16]:
model1 = RandomForestRegressor(n_estimators=1000)
model2 =AdaBoostRegressor(n_estimators=1000,learning_rate=0.01)
model3= GradientBoostingRegressor(n_estimators=1000,learning_rate=0.01)
model4 = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.01)



In [17]:
model1.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [18]:
model2.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.01, loss='linear',
                  n_estimators=1000, random_state=None)

In [19]:
model3.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [20]:
model4.fit(X_train, y_train)

[13:45:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [21]:

train_predict1=model1.predict(X_train)
test_predict1= model1.predict(X_test)

In [22]:

train_predict2=model2.predict(X_train)
test_predict2= model2.predict(X_test)

In [23]:

train_predict3=model3.predict(X_train)
test_predict3= model3.predict(X_test)

In [24]:

train_predict4=model4.predict(X_train)
test_predict4= model4.predict(X_test)

In [25]:
Y_test = np.reshape(ytest,(1, ytest.size))

In [26]:
Y_test =scaler.inverse_transform(Y_test)

In [27]:
Y_test

array([[2.21153079, 2.2089392 , 1.52635739, 0.10509743, 0.10509743,
        0.10509743, 0.10509743, 0.10509743, 0.10509743, 0.10509743,
        0.10509743, 0.10509743, 0.10509743, 0.42252864, 2.19797101,
        2.74857836, 2.74857836, 2.77935093, 3.17764741, 3.24681629,
        3.28206818, 3.28206818, 3.37756298, 3.38942514, 3.58561286,
        3.74520172, 3.75756013, 4.04013934, 4.12957771, 4.71542038,
        4.76492333]])

In [28]:
test_predict1

array([0.3787939 , 0.3743636 , 0.37437934, 0.37402807, 0.37010446,
       0.3685742 , 0.36634492, 0.3654579 , 0.36268687, 0.35892965,
       0.35847324, 0.36153203, 0.36802728, 0.36922922, 0.36227932,
       0.35993828, 0.36052521, 0.35496665, 0.35452775, 0.35964087,
       0.35727453, 0.35154451, 0.34416387, 0.34087854, 0.33841133,
       0.32799411, 0.32184754, 0.31627792, 0.31162953, 0.30908325,
       0.31777634])

In [29]:
test_predict1 = np.reshape(test_predict1,(1, test_predict1.size))
test_predict2 = np.reshape(test_predict2,(1, test_predict2.size))
test_predict3 = np.reshape(test_predict3,(1, test_predict3.size))
test_predict4 = np.reshape(test_predict4,(1, test_predict3.size))



In [30]:
test_predict1 =scaler.inverse_transform(test_predict1)
test_predict2 =scaler.inverse_transform(test_predict2)
test_predict3 =scaler.inverse_transform(test_predict3)
test_predict4 =scaler.inverse_transform(test_predict4)

In [31]:
test_predict1

array([[8.66814141, 8.56676051, 8.56712068, 8.55908232, 8.46929647,
        8.43427854, 8.3832648 , 8.36296662, 8.29955578, 8.21357717,
        8.20313311, 8.27312887, 8.42176307, 8.44926774, 8.29022956,
        8.23665823, 8.25008937, 8.12288993, 8.11284623, 8.22985251,
        8.17570218, 8.04457913, 7.87568422, 7.80050413, 7.7440458 ,
        7.50566287, 7.36500764, 7.23755525, 7.13118358, 7.07291563,
        7.2718442 ]])

In [32]:
test_predict2

array([[ 9.24561519,  9.21111417,  9.21111417,  9.21111417,  9.21111417,
         9.21111417,  9.21111417,  9.17841793,  9.16845605,  9.11523043,
         9.11419809,  9.13267713,  9.15174937,  9.16405025,  9.12392385,
         9.09286906,  9.08875395,  9.08605145,  9.0791441 ,  9.08875395,
         9.08605145,  9.1019784 ,  9.08723606,  9.0475441 ,  9.03502334,
         9.00471521,  8.97417222,  8.9022483 ,  8.9022483 ,  8.9022483 ,
        10.08843377]])

In [33]:
test_predict3

array([[8.4530988 , 8.32724072, 8.32191827, 8.3518643 , 8.32191827,
        8.33602586, 8.25308724, 8.20800961, 8.05940702, 7.59964556,
        7.56473952, 7.76447733, 7.96266612, 8.02879271, 7.91182636,
        7.91368875, 7.91530131, 7.83276934, 7.81799144, 8.26975053,
        8.02454827, 8.25510671, 8.31193583, 8.14331089, 7.833995  ,
        7.64384949, 7.39563258, 6.99664542, 6.97774667, 6.97774667,
        7.38122881]])

In [34]:
test_predict4

array([[ 9.067877 ,  9.067877 ,  9.067877 ,  9.069323 ,  9.067877 ,
         9.067877 ,  9.067877 ,  9.261084 ,  9.066464 ,  8.847291 ,
         8.845845 ,  8.845845 ,  9.142316 ,  9.142316 ,  8.946251 ,
         8.95466  ,  9.19407  ,  8.946251 ,  8.95466  , 13.7949705,
         9.202479 ,  9.19407  ,  8.946251 ,  8.946251 ,  8.946251 ,
         9.073375 ,  9.082456 ,  9.082456 ,  9.046171 ,  9.046171 ,
         9.046171 ]], dtype=float32)

In [35]:
#Y_test = Y_test.transpose()
RMSE1 = math.sqrt(mean_squared_error(Y_test,test_predict1))
RMSE1

6.2606777825777575

In [36]:
#Y_test = Y_test.transpose()
RMSE2 = math.sqrt(mean_squared_error(Y_test,test_predict2))
RMSE2

7.188679349661502

In [37]:
#Y_test = Y_test.transpose()
RMSE3 = math.sqrt(mean_squared_error(Y_test,test_predict3))
RMSE3

6.061592498796213

In [38]:
#Y_test = Y_test.transpose()
RMSE4 = math.sqrt(mean_squared_error(Y_test,test_predict4))
RMSE4

7.263860180760081

In [39]:

from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict1)
print('MAE: %f' % mae)

MAE: 5.933863


In [40]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict2)
print('MAE: %f' % mae)

MAE: 6.994827


In [41]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict3)
print('MAE: %f' % mae)

MAE: 5.766682


In [42]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict4)
print('MAE: %f' % mae)

MAE: 7.052964
